In [105]:
import pandas as pd
import numpy as np

In [106]:
bet_mgm_df = pd.read_csv('data/bet_mgm_mlb.csv')
caesars_df = pd.read_csv('data/caesars_mlb.csv')
draft_kings_df = pd.read_csv('data/draft_kings_mlb.csv')
events_df = pd.read_csv('data/events_mlb.csv')

In [107]:
sportsbooks = [(bet_mgm_df, 'Bet MGM'), (caesars_df, 'Caesars'), (draft_kings_df, 'Draft Kings')]

In [108]:
bet_mgm_df.head(6)

,Team,Spread,Total,Moneyline,Event ID
0,White Sox,-4.5 +120,O 9.5 -140,-1600.0,d3bac1cb-103f-439d-9a6a-efb0dc865e14
1,Orioles,+4.5 -155,U 9.5 +110,825.0,d3bac1cb-103f-439d-9a6a-efb0dc865e14
2,Nationals,NaN,NaN,NaN,6640ceb8-4c9f-4f45-85d5-e900c002e508
3,Marlins,NaN,NaN,NaN,6640ceb8-4c9f-4f45-85d5-e900c002e508
4,Astros,NaN,NaN,NaN,d99185a8-2be7-4a89-8496-507b211a108a
5,Reds,NaN,NaN,NaN,d99185a8-2be7-4a89-8496-507b211a108a


In [109]:
caesars_df.head(6)

,Team,Spread,Total,Moneyline,Event ID
0,White Sox,NaN,NaN,NaN,d3bac1cb-103f-439d-9a6a-efb0dc865e14
1,Orioles,NaN,NaN,NaN,d3bac1cb-103f-439d-9a6a-efb0dc865e14
2,Astros,NaN,NaN,NaN,d99185a8-2be7-4a89-8496-507b211a108a
3,Reds,NaN,NaN,NaN,d99185a8-2be7-4a89-8496-507b211a108a
4,Nationals,NaN,NaN,NaN,6640ceb8-4c9f-4f45-85d5-e900c002e508
5,Marlins,NaN,NaN,NaN,6640ceb8-4c9f-4f45-85d5-e900c002e508


In [110]:
draft_kings_df.head(6)

,Team,Spread,Total,Moneyline,Event ID
0,White Sox,-4.5+120,O 9.5−135,−1450,d3bac1cb-103f-439d-9a6a-efb0dc865e14
1,Orioles,+4.5−154,U 9.5+105,+750,d3bac1cb-103f-439d-9a6a-efb0dc865e14
2,Nationals,-1.5+250,O 4.5+100,+120,6640ceb8-4c9f-4f45-85d5-e900c002e508
3,Marlins,+1.5−345,U 4.5−130,−154,6640ceb8-4c9f-4f45-85d5-e900c002e508
4,Astros,+8.5−110,O 15.5−135,NaN,d99185a8-2be7-4a89-8496-507b211a108a
5,Reds,-8.5−120,U 15.5+105,NaN,d99185a8-2be7-4a89-8496-507b211a108a


In [111]:
events_df

,Event ID,Team One,Team Two
0,d3bac1cb-103f-439d-9a6a-efb0dc865e14,White Sox,Orioles
1,6640ceb8-4c9f-4f45-85d5-e900c002e508,Nationals,Marlins
2,d99185a8-2be7-4a89-8496-507b211a108a,Astros,Reds
3,ceb8804e-7b9b-4aa8-b9c7-8356c0bd4b99,Twins,Rays
4,a0ac983d-fd63-4417-b083-c73a2a26bc6c,Red Sox,Mets
5,818cdd89-0865-47e5-91ac-861100beb87d,Rockies,Braves
6,c2295c3e-8680-4105-b862-0bee39c0cb8e,Pirates,Cubs
7,e5669d8f-8fc8-4041-9c11-c6db82349215,Guardians,Royals
8,ee9dd335-af73-4378-96d9-d42e6136ae47,Cardinals,Brewers
9,a9b9439e-f716-4804-8a7d-46285f619b6f,Yankees,Rangers


In [112]:
def clean_moneyline(ml_value):
	if isinstance(ml_value, str):
		ml_value = ml_value.replace('−', '-').replace('+', '')  # Clean up special minus or plus signs
	return int(ml_value)  # Convert to integer

def implied_probability(moneyline):
	if moneyline < 0:
		return -moneyline / (-moneyline + 100)
	else:
		return 100 / (moneyline + 100)

def check_arbitrage(team_one_ml, team_two_ml):
	# Get the lowest implied probabilities for both teams
	best_team_one_prob = min([implied_probability(ml) for ml, book in team_one_ml])
	best_team_two_prob = min([implied_probability(ml) for ml, book in team_two_ml])
	
	# Sum the best probabilities
	total_implied_prob = best_team_one_prob + best_team_two_prob
	
	# Check for arbitrage
	if total_implied_prob < 1:
		print("Arbitrage opportunity exists!", total_implied_prob)
	else:
		print("No arbitrage opportunity.", total_implied_prob)


In [121]:
def has_arbitrage(event_id):
    row = events_df[events_df['Event ID'] == event_id]
    team_one = row['Team One'].item()
    team_two = row['Team Two'].item()
    
    print(team_one, team_two)

    team_one_ml = []
    team_two_ml = []
    
    for book, book_name in sportsbooks:
        team_one_ml_raw = book[(book['Event ID'] == event_id) & (book['Team'] == team_one)]['Moneyline']
        team_two_ml_raw = book[(book['Event ID'] == event_id) & (book['Team'] == team_two)]['Moneyline']

        if not team_one_ml_raw.empty and not pd.isna(team_one_ml_raw.item()):
            team_one_ml.append((clean_moneyline(team_one_ml_raw.item()), book_name))
        if not team_two_ml_raw.empty and not pd.isna(team_two_ml_raw.item()):
            team_two_ml.append((clean_moneyline(team_two_ml_raw.item()), book_name))
    
    check_arbitrage(team_one_ml, team_two_ml)

	


In [122]:
for index, event in events_df.iterrows():
	event_id = event['Event ID']
	# team_one = event['Team One']
	# team_two = event['Team Two']
	has_arbitrage(event_id)

	

White Sox Orioles
No arbitrage opportunity. 1.0435919790758499
Nationals Marlins
No arbitrage opportunity. 1.0608446671438798
Astros Reds


ValueError: can only convert an array of size 1 to a Python scalar